In [1]:
import pandas as pd
import numpy as np 
import unicodedata
import re
from difflib import get_close_matches

In [2]:
df = pd.read_csv("train.csv", low_memory=False)
df

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,...,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id
0,2014,52.0,Erkek,4/6/1994,"Altindag, Ankara",Ankara,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,...,Diğer,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,0
1,2014,30.0,Erkek,6/11/1993,Üsküdar,İstanbul,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Hayır,...,0,Hayır,NaN,Evet,Evet,Bilkent Cyberparkta bir şirkette Türkiye nin i...,NaN,NaN,NaN,1
2,2014,18.0,Erkek,1/15/1986,Samsun,İstanbul,ULUSLARARASI KIBRIS ÜNİVERSİTESİ,Özel,100.0,Hayır,...,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,2
3,2014,40.0,Erkek,6/4/1991,Diyarbakır,İstanbul,İSTANBUL ŞEHİR ÜNİVERSİTESİ,Özel,100.0,Evet,...,0,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,3
4,2014,24.0,Erkek,2 Kasim 1992,Ankara/Altındağ,Ankara,TURGUT ÖZAL ÜNİVERSİTESİ,Özel,100.0,Evet,...,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,Erkek,18.06.2003 00:00,İstanbul,İstanbul,İstanbul Aydın Üniversitesi,Özel,NaN,Evet,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65120
65121,2022,0.0,Erkek,18.04.2002 00:00,Hatay,Muğla,Muğla Sıtkı Koçman Üniversitesi,Devlet,NaN,Hayır,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65121
65122,2022,0.0,Erkek,23.10.1999 00:00,Mersin,Mersin,Süleyman Demirel Üniversitesi,Devlet,NaN,Hayır,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65122
65123,2022,0.0,Erkek,17.11.2000 00:00,Samsun,Samsun,Karabük Üniversitesi,Devlet,NaN,Hayır,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65123


In [3]:
df_raw=df.copy()

> Görüldüğü gibi çok fazla boş değerimiz var. Dolayısıyla, ilk önce <mark>boş değer analizi</mark> yapılacak.

In [4]:
df_nulls = pd.DataFrame(df.isnull().sum().sort_values(ascending=False), columns=['Kayıp veri miktarı'])
df_nulls['% Kayıp veri miktarı'] = df.isnull().sum().sort_values(ascending=False)/len(df)
df_nulls

,Kayıp veri miktarı,% Kayıp veri miktarı
"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",64706,0.993566
Lise Bolum Diger,63599,0.976568
Lise Adi Diger,60392,0.927324
Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,50526,0.775831
Hangi STK'nin Uyesisiniz?,49518,0.760353
Uye Oldugunuz Kulubun Ismi,48691,0.747655
Baska Kurumdan Aldigi Burs Miktari,44162,0.678111
Burs Aldigi Baska Kurum,43977,0.675271
Burslu ise Burs Yuzdesi,42685,0.655432
Daha Once Baska Bir Universiteden Mezun Olmus,37345,0.573436


In [5]:
df['Universite Not Ortalamasi'].unique()

array(['3.50-3', '3.00-2.50', nan, '2.50 ve altı', '3.00 - 4.00',
       '3.50 - 4.00', '3.00 - 3.50', '2.50 -3.00', 'Ortalama bulunmuyor',
       'ORTALAMA BULUNMUYOR', '4-3.5', '2.50 - 3.00', '2.00 - 2.50',
       '1.00 - 2.50', 'Not ortalaması yok', '4.0-3.5', '3.00 - 3.49',
       '2.50 - 2.99', '1.80 - 2.49', 'Hazırlığım', '0 - 1.79'],
      dtype=object)

> Veri setinde yinelenen değer <mark>bulunmamakta</mark>.

In [6]:
df[df.duplicated()] 

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,...,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id


> Veri setimiz hem <mark>kategorik</mark> hem de <mark>numerik</mark> değerlerden oluşmaktadır.
Numerik değer ve kategorik değerlerin analizi yapılacak.

In [7]:
df.dtypes

Basvuru Yili                                                   int64
Degerlendirme Puani                                          float64
Cinsiyet                                                      object
Dogum Tarihi                                                  object
Dogum Yeri                                                    object
Ikametgah Sehri                                               object
Universite Adi                                                object
Universite Turu                                               object
Burslu ise Burs Yuzdesi                                      float64
Burs Aliyor mu?                                               object
Bölüm                                                         object
Universite Kacinci Sinif                                      object
Universite Not Ortalamasi                                     object
Daha Once Baska Bir Universiteden Mezun Olmus                 object
Lise Adi                          

In [8]:
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].str.lower()

In [9]:
df =  df.drop(['Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite'], axis=1)

In [10]:
df.loc[df['Uye Oldugunuz Kulubun Ismi'].notnull(), 'Uye Oldugunuz Kulubun Ismi'] = 'evet'
df.loc[df['Uye Oldugunuz Kulubun Ismi'].isnull(), 'Uye Oldugunuz Kulubun Ismi'] = 'hayır'

In [11]:
df['Burslu ise Burs Yuzdesi'] = df['Burslu ise Burs Yuzdesi'].fillna(np.float64(0))

In [12]:
df['Daha Once Baska Bir Universiteden Mezun Olmus'] = df['Daha Once Baska Bir Universiteden Mezun Olmus'].fillna('hayır')

In [13]:
df['Burs Aldigi Baska Kurum'] = df['Burs Aldigi Baska Kurum'].fillna('hayır')
df.loc[df['Burs Aldigi Baska Kurum'].notnull(), 'Burs Aldigi Baska Kurum'] = 'evet'
df.loc[df['Burs Aldigi Baska Kurum'].isnull(), 'Burs Aldigi Baska Kurum'] = 'hayır'

In [14]:
df = df.drop(['Ingilizce Seviyeniz?'], axis=1)

In [15]:
replace_roles = {
    "kaptan / li̇der": "kaptan",
    "lider/kaptan": "kaptan",
    "diğer": "di̇ğer",
    "0": '-',
}

df['Spor Dalindaki Rolunuz Nedir?'] = df['Spor Dalindaki Rolunuz Nedir?'].replace(replace_roles)
df['Spor Dalindaki Rolunuz Nedir?'] = df['Spor Dalindaki Rolunuz Nedir?'].fillna('-')
df['Spor Dalindaki Rolunuz Nedir?'].unique()

array(['di̇ğer', '-', 'kaptan', 'bireysel', 'takım oyuncusu'],
      dtype=object)

In [16]:
df['Stk Projesine Katildiniz Mi?'] = df['Stk Projesine Katildiniz Mi?'].fillna('hayır')

In [17]:
df['Aktif olarak bir STK üyesi misiniz?'] = df['Aktif olarak bir STK üyesi misiniz?'].fillna('hayır')

In [18]:
df['Girisimcilikle Ilgili Deneyiminiz Var Mi?'] = df['Girisimcilikle Ilgili Deneyiminiz Var Mi?'].fillna('hayır')

In [19]:
# sorulmayan seneleri bul
df['Ingilizce Biliyor musunuz?'] = df['Ingilizce Biliyor musunuz?'].fillna('hayır')

In [20]:
test =  pd.read_csv("test_x.csv")
test['Lise Mezuniyet Notu'].unique()

array(['50 - 74', '75 - 100', '25 - 49'], dtype=object)

In [21]:
df['Lise Mezuniyet Notu'].unique()

array(['3.50-3', '3.00 - 4.00', '3.00-2.50', '2.50 ve altı', '4.00-3.50',
       '3.50-3.00', nan, '69-55', '100-85', '84-70', '54-45', '44-0',
       '75 - 100', '50 - 75', 'not ortalaması yok', '25 - 50', '0 - 25',
       '50 - 74', '25 - 49', '0 - 24'], dtype=object)

In [22]:
df['Lise Mezuniyet Notu'] =  df['Lise Mezuniyet Notu'].fillna('not ortalaması yok')

lise_notes = {
    'not ortalaması yok': '0 - 24',
    '2.50 ve altı': '0 - 24',
    '0 - 24': '0 - 24',
    '0 - 25': '0 - 24',
    '44-0': '0 - 24',
    '25 - 49': '25 - 49',
    '25 - 50': '25 - 49',
    '54-45': '25 - 49',
    '69-55': '50 - 74',
    '3.00-2.50': '50 - 74',
    '50 - 74': '50 - 74',
    '50 - 75' : '50 - 74',
    '3.50-3.00': '50 - 74',
    '3.50-3': '75 - 100',
    '84-70': '75 - 100',
    '75 - 100': '75 - 100',
    '100-85': '75 - 100',
    '3.00 - 4.00': '75 - 100',
    '4.00-3.50': '75 - 100',
}

df['Lise Mezuniyet Notu'] = df['Lise Mezuniyet Notu'].replace(lise_notes)
df['Lise Mezuniyet Notu'].unique()

array(['75 - 100', '50 - 74', '0 - 24', '25 - 49'], dtype=object)

In [23]:
test['Universite Not Ortalamasi'].unique()

array(['2.50 - 2.99', 'Hazırlığım', '3.00 - 3.49', '1.80 - 2.49',
       '3.50 - 4.00', '0 - 1.79'], dtype=object)

In [24]:
uni_notes = {
    'ortalama bulunmuyor': '0 - 1.79',
    'not ortalaması yok': '0 - 1.79',
    'hazırlığım': 'hazırlığım',
    '1.00 - 2.50': '1.80 - 2.49',
    '2.50 ve altı':  '1.80 - 2.49',
    '1.80 - 2.49' : '1.80 - 2.49',
    '0 - 1.79': '0 - 1.79',
    '2.00 - 2.50': '1.80 - 2.49',
    '2.50 - 2.99': '2.50 - 2.99',
    '2.50 - 3.00': '2.50 - 2.99',
    '2.50 -3.00': '2.50 - 2.99',
    '3.00-2.50': '2.50 - 2.99',
    '3.50-3': '3.00 - 3.49',
    '3.00 - 3.50': '3.00 - 3.49',
    '3.00 - 3.49': '3.00 - 3.49',
    '3.00 - 4.00': '3.00 - 4.00',
    '3.50 - 4.00': '3.50 - 4.00',
    '4-3.5': '3.50 - 4.00',
    '4.0-3.5': '3.50 - 4.00',
}

df['Universite Not Ortalamasi'] = df['Universite Not Ortalamasi'].fillna('not ortalaması yok')
df['Universite Not Ortalamasi'] = df['Universite Not Ortalamasi'].replace(uni_notes)
df['Universite Not Ortalamasi'].unique()

array(['3.00 - 3.49', '2.50 - 2.99', '0 - 1.79', '1.80 - 2.49',
       '3.00 - 4.00', '3.50 - 4.00', 'hazırlığım'], dtype=object)

In [25]:
df = df.dropna(subset=['Universite Turu'])

In [26]:
df.loc[df['Universite Adi'].isna() & df['Bölüm'].notnull(), 'Universite Adi'] = 'diğer'
df.loc[df['Universite Adi'].isna() & df['Universite Turu'].notnull(), 'Universite Adi'] = 'diğer'

In [27]:
df['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'] = df['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'].fillna('hayır')

In [28]:
df['Profesyonel Bir Spor Daliyla Mesgul musunuz?'] = df['Profesyonel Bir Spor Daliyla Mesgul musunuz?'].fillna('hayır')

In [29]:
df['Universite Kacinci Sinif'] = df['Universite Kacinci Sinif'].fillna(0)

replace_grade = {
    'tez': 'yüksek lisans'
}

df['Universite Kacinci Sinif'] = df['Universite Kacinci Sinif'].replace(replace_grade)

In [30]:
df = df.drop(['Bölüm'],axis=1)

In [31]:
replace_lise_turu = {
    "özel": "özel lise",
    "özel lisesi": "özel lise",
    "meslek": "meslek lisesi",
}
df['Lise Turu'] = df['Lise Turu'].replace(replace_lise_turu)
print(df['Lise Turu'].unique())
# boş değerler
df.loc[df['Lise Turu'].isnull() & 
    df['Lise Adi'].str.contains("anadolu", na=False), "Lise Turu"] = "anadolu lisesi"

df.loc[df['Lise Turu'].isnull() & 
    df['Lise Adi'].str.contains("fen", na=False), "Lise Turu"] = "fen lisesi"

df.loc[df['Lise Turu'].isnull() & 
    df['Lise Adi'].str.contains("açık|acık|acik|acık|açik", na=False), "Lise Turu"] = "düz lise"

df.loc[df['Lise Turu'].isnull() & 
    df['Lise Adi'].str.contains("imam|ihl|i.h.l|ih", na=False), "Lise Turu"] = "i̇mam hatip lisesi"

df.loc[df['Lise Turu'].isnull() & 
    df['Lise Adi'].str.contains("meslek|mtal|m.t.a.l|mes|mes.|atml|a.t.m.l|mesleki", na=False), "Lise Turu"] = "meslek lisesi"

# devlet ve diğer olanların 'Lise Adi'na bak
df.loc[((df['Lise Turu']=='devlet') | (df['Lise Turu']=='diğer')) & 
    df['Lise Adi'].str.contains('anadolu'), 'Lise Turu'] = 'anadolu lisesi'

df.loc[((df['Lise Turu']=='devlet') | (df['Lise Turu']=='diğer')) & 
    df['Lise Adi'].str.contains('fen'), 'Lise Turu'] = 'fen lisesi'

df.loc[((df['Lise Turu']=='devlet') | (df['Lise Turu']=='diğer')) & 
    df['Lise Adi'].str.contains('açık|acık|acik|acık|açik'), 'Lise Turu'] = 'düz lise'

df.loc[((df['Lise Turu']=='devlet') | (df['Lise Turu']=='diğer')) & 
    df['Lise Adi'].str.contains('imam|ihl|i.h.l|ih'), 'Lise Turu'] = 'i̇mam hatip lisesi'

df.loc[((df['Lise Turu']=='devlet') | (df['Lise Turu']=='diğer')) & 
    df['Lise Adi'].str.contains('meslek|mtal|m.t.a.l|mes|mes.|atml|a.t.m.l|mesleki'), 'Lise Turu'] = 'meslek lisesi'

df = df.dropna(subset=['Lise Turu'])

['anadolu lisesi' 'meslek lisesi' 'fen lisesi' 'özel lise' 'düz lise'
 'i̇mam hatip lisesi' nan 'diğer' 'devlet']


In [32]:
df['Lise Turu'].isna().sum()

np.int64(0)

In [33]:
df = df.drop(["Lise Adi", "Lise Adi Diger"], axis=1)

replace_bolum = {
    'düz lise bölüm yok': 'alan yok',
    'alansız': 'alan yok',
    'alan ayrımı yok': 'alan yok',
    'alan ayrımı.olmayan lise': 'alan yok',
    'bölüm ayrımı olmayanlar': 'alan yok',
    'bölüm ayrımı bulunmamaktadır.': 'alan yok',
    'alan ayrimi olmayanlar': 'alan yok',
    '-': 'alan yok',
    'yok': 'alan yok',
    'alan bulunmayan özel lise': 'alan yok',
    '': 'alan yok',
    'alansız': 'alan yok',
    'bölüm yok': 'alan yok',
    'alan ayrimi olmayanlar': 'alan yok',
    '_': 'alan yok',
    '.': 'alan yok',
    'bölüm yoktu': 'alan yok',
    'bolum yok': 'alan yok',
    'bölüm ayrımı olmayan': 'alan yok'
}

df['Lise Bolumu'] = df['Lise Bolumu'].replace(replace_bolum)

df.loc[df['Lise Bolumu'].isnull() & df['Lise Bolum Diger'].notnull(), 'Lise Bolumu'] = df['Lise Bolum Diger']

df = df.dropna(subset=['Lise Bolumu'])

filtered_sayisal = df['Lise Bolumu'].str.contains("fen|mf|sayısal|sayisal|fm|m-f")
filtered_ea = df['Lise Bolumu'].str.contains("eşit ağırlık|esit agırlık|ea|tm|türkçe-matematik|türkçe matematik|turkce matematik|eşit agırlık|esıt agırlık|türkçe-mat|esi̇t agirlik|eşit agırlık|eşit agırlık|türkçe-mat|turkce-matematik|eşi̇t agirlik|türkçe -matematik|e-a|türkçe ve matematik|eşi̇t ağirlik|eşitağırlık|türkçe- matematik|turkce-matematık|turkce ve matematık|matematik-türkçe|turkce matemati̇k|esitagirlik|türkçe - matematik|t-m|t.m")
filtered_ts = df['Lise Bolumu'].str.contains("sözel|sozel|türkçe-sosyal|türkçe sosyal|sosyal bilimler|ts|t/s")
filtered_dil = df['Lise Bolumu'].str.contains("dil|ingilizce|yabancı di̇l")

combine = pd.concat([filtered_sayisal, filtered_ea, filtered_ts, filtered_dil])

filtered_another = df[~df['Lise Bolumu'].isin(combine)]

df.loc[df['Lise Turu'] == 'meslek lisesi', 'Lise Bolumu'] = 'diğer'
df.loc[filtered_sayisal, 'Lise Bolumu'] = 'sayısal'
df.loc[filtered_ea, 'Lise Bolumu'] = 'eşit ağırlık'
df.loc[filtered_ts, 'Lise Bolumu'] = 'sözel'
df.loc[filtered_dil, 'Lise Bolumu'] = 'dil'

invalid_ids = df.loc[~df['Lise Bolumu'].isin(['diğer', 'sayısal', 'eşit ağırlık', 'sözel', 'dil']), 'id'].tolist()
df = df[~df['id'].isin(invalid_ids)]

df = df.drop(['Lise Bolum Diger'], axis=1)
df['Lise Bolumu'].unique()

In [34]:
df['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'] = df['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'].fillna('hayır')

filtered = df['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'].str.contains('girişimim yok|yer almak cok isterim|görev almayı çok isterim|deneyimim yok', na=False)

df.loc[filtered, 'Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'] = 'hayır'

df.loc[~filtered, 'Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'] = 'evet'

df['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'].unique()


array(['evet', 'hayır'], dtype=object)

In [35]:
df = df.drop(['Hangi STK\'nin Uyesisiniz?'], axis=1)
df.shape

(64414, 38)

In [36]:
baba_egitim = {
    'üniversite mezunu': 'üniversite',
    'üni̇versi̇te': 'üniversite',
    'yüksek lisans / doktara': 'yüksek lisans / doktora',
    'i̇lkokul mezunu': 'i̇lkokul',
    'ortaokul mezunu': 'ortaokul',
    'lise mezunu': 'lise',
    'li̇se': 'lise',
    '0': 'eğitimi yok',
    'eği̇ti̇m yok': 'eğitimi yok',
    'eğitim yok': 'eğitimi yok',
    'yüksek li̇sans': 'yüksek lisans'
}

df['Baba Egitim Durumu'] = df['Baba Egitim Durumu'].fillna('eğitimi yok')

df['Baba Egitim Durumu'] = df['Baba Egitim Durumu'].replace(baba_egitim)

df['Baba Egitim Durumu'].unique()

array(['üniversite', 'yüksek lisans / doktora', 'i̇lkokul', 'eğitimi yok',
       'ortaokul', 'lise', 'yüksek lisans', 'doktora'], dtype=object)

In [37]:
df['Anne Egitim Durumu'] = df['Anne Egitim Durumu'].fillna('eğitimi yok')

df['Anne Egitim Durumu'] = df['Anne Egitim Durumu'].replace(baba_egitim)

df['Anne Egitim Durumu'].unique()


array(['üniversite', 'i̇lkokul', 'eğitimi yok', 'lise', 'ortaokul',
       'yüksek lisans / doktora', 'doktora', 'yüksek lisans'],
      dtype=object)

In [38]:
df['Anne Sektor'].unique()

array(['0', 'kamu', 'özel sektör', 'diğer', nan, 'di̇ğer', '-'],
      dtype=object)

In [39]:
anne_sektor = {
    'diğer': 'di̇ğer',
    '0': '-',
}
df["Anne Calisma Durumu"] = df['Anne Sektor'].apply(lambda x: "evet" if x != "-" else "hayır")


In [40]:
df.loc[(df["Anne Calisma Durumu"]=='evet') & ((df['Anne Sektor']=='-') | (df['Anne Sektor'].isna()) | (df['Anne Sektor']=='0')), 'Anne Sektor']= 'diğer'
df['Anne Sektor'] = df['Anne Sektor'].fillna("-")
df['Anne Sektor'] = df['Anne Sektor'].replace(anne_sektor)

In [41]:
df["Baba Calisma Durumu"] = df['Baba Sektor'].apply(lambda x: "evet" if x != "-" else "hayır")

In [42]:
df.loc[(df["Baba Calisma Durumu"]=='evet') & ((df['Baba Sektor']=='-') | (df['Baba Sektor'].isna()) | (df['Baba Sektor']=='0')), 'Baba Sektor']= 'diğer'

df['Baba Sektor'] = df['Baba Sektor'].fillna("-")
df['Baba Sektor'] = df['Baba Sektor'].replace(anne_sektor)

In [43]:
df['Cinsiyet'] = df['Cinsiyet'].fillna('belirtmek istemiyorum')

In [44]:
df = df.dropna(subset=['Degerlendirme Puani'])

In [45]:
tr = pd.read_csv("il_ilce.csv")

In [46]:
for col in tr.columns:
    if tr[col].dtype == 'object':
        tr[col] = tr[col].str.lower()
tr["il"].unique()

array(['adana', 'adıyaman', 'afyonkarahisar', 'aksaray', 'amasya',
       'ankara', 'antalya', 'ardahan', 'artvin', 'aydın', 'ağrı',
       'balıkesir', 'bartın', 'batman', 'bayburt', 'bilecik', 'bingöl',
       'bitlis', 'bolu', 'burdur', 'bursa', 'denizli', 'diyarbakır',
       'düzce', 'edirne', 'elazığ', 'erzincan', 'erzurum', 'eskişehir',
       'gaziantep', 'giresun', 'gümüşhane', 'hakkâri', 'hatay', 'isparta',
       'iğdır', 'kahramanmaraş', 'karabük', 'karaman', 'kars',
       'kastamonu', 'kayseri', 'kilis', 'kocaeli', 'konya', 'kütahya',
       'kırklareli', 'kırıkkale', 'kırşehir', 'malatya', 'manisa',
       'mardin', 'mersin', 'muğla', 'muş', 'nevşehir', 'niğde', 'ordu',
       'osmaniye', 'rize', 'sakarya', 'samsun', 'siirt', 'sinop', 'sivas',
       'tekirdağ', 'tokat', 'trabzon', 'tunceli', 'uşak', 'van', 'yalova',
       'yozgat', 'zonguldak', 'çanakkale', 'çankırı', 'çorum',
       'i̇stanbul', 'i̇zmir', 'şanlıurfa', 'şırnak', 'kosova', 'bakü',
       'kıbrıs', 'alma

In [47]:
tr2 = tr.set_index('ilce')

def process_cities(df, data, fill_data, data_map):
    
    df[data] = df[data].str.replace('ı', 'i', regex=False)
    df.loc[df[data].isnull() & df[fill_data].notnull(), data] = df[fill_data]
    
    df = df.dropna(subset=[data])

    df[data] = df[data].apply(
        lambda x: unicodedata.normalize('NFKD', x.strip().lower()).encode('ASCII', 'ignore').decode('ASCII')
    )

    df[data] = df[data].apply(
        lambda x: re.split(r'[\/\-,~ ]', x)[-1].strip()
    )

    df[data] =  df[data].replace(data_map)

    ids_to_drop = []

    for city, id_ in zip(df[data], df["id"]):
        if city not in tr["il"]: # verisetinde olmamasının nedini yazım yanlışlığı olabilir
            match_city = get_close_matches(city, tr2["il"]) # yazımı kontrol et
            if match_city: # match bulundu ise
                df.loc[df["id"]==id_, data] = match_city[0] # yazımı düzelt
            else: # şehir match'ı bulunmadı 
                match_district = get_close_matches(city, tr["ilce"]) # tr'deki ilçeleri kontrol et
                if match_district: # ilçe varsa
                    province = tr.loc[tr['ilce'] == match_district[0], 'il'].values  
                    if len(province) > 0: # ilçe verisetinde var ise
                        df.loc[df["id"] == id_, data] = province[0]
                    else: # ilçe, veri setinde yok ise
                        print("TR veri setinde yok", id_, "   ", city)
                else: # ilçe match'ı bulunmadıysa
                    ids_to_drop.append(id_)
                    print("MATCH YOK: ", id_, "   ", city)
            
    df = df[~df["id"].isin(ids_to_drop)]
    return df


In [48]:
replace_ikametgah = {
    "ş.urfa": "şanlıurfa",
    "k.maraş": "kahramanmaraş"
}

df = process_cities(df, 'Ikametgah Sehri', 'Lise Sehir', replace_ikametgah)

/tmp/ipykernel_26379/3792083127.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[data] = df[data].apply(
/tmp/ipykernel_26379/3792083127.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[data] = df[data].apply(
/tmp/ipykernel_26379/3792083127.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

MATCH YOK:  692     
MATCH YOK:  850     
MATCH YOK:  1008     
MATCH YOK:  5135     
MATCH YOK:  6516     
MATCH YOK:  6517     
MATCH YOK:  6767     
MATCH YOK:  9550     
MATCH YOK:  10816     gothamcity
MATCH YOK:  11344     gdr
MATCH YOK:  13255     gothamcity
MATCH YOK:  13919     openairequestfailedtimeout
MATCH YOK:  14025     gothamcity
MATCH YOK:  14238     gothamcity
MATCH YOK:  14378     openairequestfailedtimeout
MATCH YOK:  14983     gdr
MATCH YOK:  15698     
MATCH YOK:  15699     
MATCH YOK:  15992     
MATCH YOK:  16049     
MATCH YOK:  16425     
MATCH YOK:  16598     
MATCH YOK:  17014     
MATCH YOK:  17167     
MATCH YOK:  17212     
MATCH YOK:  17535     
MATCH YOK:  18034     
MATCH YOK:  18210     
MATCH YOK:  18291     
MATCH YOK:  18608     
MATCH YOK:  18820     
MATCH YOK:  18904     
MATCH YOK:  19182     
MATCH YOK:  19306     
MATCH YOK:  19390     
MATCH YOK:  19534     
MATCH YOK:  19582     
MATCH YOK:  19769     
MATCH YOK:  19981     
MATCH YOK:  203

In [49]:
replace_lise_sehir = {
    "ist." : "istanbul",
    "istanbulkucukcekmece": "istanbul",
    "ş.urfa": "şanlıurfa",
    "k.maraş": "kahramanmaraş"
}

df = process_cities(df, 'Lise Sehir', 'Ikametgah Sehri', replace_lise_sehir)
df = df[~df['Lise Sehir'].isin(['', 'shkoder', 'asdasd'])]

MATCH YOK:  833     
MATCH YOK:  6568     
MATCH YOK:  10431     shkoder
MATCH YOK:  10667     asdasd


KeyboardInterrupt: 

In [ ]:
replace_dy = {
    "ş.urfa": "şanlıurfa",
    "k.maras": "kahramanmaraş",
    "k.maraş": "kahramanmaraş",
    "ist." : "istanbul",
    "eyup": "eyüpsultan",
    "gumushacikoy": "gümüşhacıköy",
    "cicekdagi": "çiçekdağı",
    "sevastopol": "sivastopol",
    "yigilca": "yığılca"
}

df = process_cities(df, 'Dogum Yeri', 'Ikametgah Sehri', replace_dy)

In [ ]:
replace_sentence = {
    'kardeş sayısı 1 ek bilgi aile hk. anne vefat': 1
}

df["Kardes Sayisi"] = df["Kardes Sayisi"].replace(replace_sentence)

df['Kardes Sayisi'] = pd.to_numeric(df['Kardes Sayisi'])

df["Kardes Sayisi"] = df["Kardes Sayisi"].fillna(0)

df["Kardes Sayisi"] = df["Kardes Sayisi"].astype(int)

df["Kardes Sayisi"].unique()

In [ ]:
df['Basvuru Yili'] = pd.to_datetime(df['Basvuru Yili'], format='%Y')

In [ ]:
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='\\.', value="/", regex=True)

In [ ]:
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='ocak', value='/01/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='subat', value='/02/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='mart', value='/03/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='nisan', value='/04/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='mayis', value='/05/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='haziran', value='/06/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='temmuz', value='/07/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='ağustos', value='/08/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='eylül', value='/09/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='ekim', value='/10/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='kasim', value='/11/', regex=True)
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='aralik', value='/12/', regex=True)

In [ ]:
df['Dogum Tarihi']= df['Dogum Tarihi'].replace(to_replace='\\_', value="", regex=True)

In [ ]:
df['Dogum Tarihi'] = pd.to_datetime(df['Dogum Tarihi'], format='mixed', errors='coerce')

In [ ]:
df.loc[df['Dogum Tarihi'].isnull() == True].shape

In [ ]:
df.dropna(subset='Dogum Tarihi', inplace=True)

In [ ]:
df['yas']=df['Basvuru Yili'].dt.year - df['Dogum Tarihi'].dt.year
df["yas"] = df["yas"].astype(int)

In [ ]:
df['Baska Kurumdan Aldigi Burs Miktari']

In [ ]:
df["Baska Kurumdan Aldigi Burs Miktari"].fillna("-")

In [ ]:
df=df.drop(df.loc[~((df['Baska Kurumdan Aldigi Burs Miktari'] == '1000₺ ve üstü')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] ==  '500₺ - 999₺')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] ==  '500 ve üzeri')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] == '0 - 499₺')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] == '-')
                ) & (
                    (df["Baska Kurumdan Aldigi Burs Miktari"].str.contains('%', na=False, case=False)) 
                    | (df["Baska Kurumdan Aldigi Burs Miktari"].str.contains('yüzde', na=False, case=False))
                    | (df["Baska Kurumdan Aldigi Burs Miktari"].str.contains('kyk', na=False, case=False))
                    | (df["Baska Kurumdan Aldigi Burs Miktari"].str.contains('her sene', na=False, case=False))
                    | (df["Baska Kurumdan Aldigi Burs Miktari"].str.contains('\\+', na=False, case=False))
                    | (df["Baska Kurumdan Aldigi Burs Miktari"].str.contains('artı', na=False, case=False))
                    | (df["Baska Kurumdan Aldigi Burs Miktari"].str.contains('arti', na=False, case=False))
                    | (df["Baska Kurumdan Aldigi Burs Miktari"].str.contains('ve', na=False, case=False))
                    | (df["Baska Kurumdan Aldigi Burs Miktari"].str.contains(',', na=False, case=False))
                    | (df["Baska Kurumdan Aldigi Burs Miktari"].str.contains('x', na=False, case=False))
                )].index)


In [ ]:
df.loc[
                        ~((df['Baska Kurumdan Aldigi Burs Miktari'] == '1000₺ ve üstü')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] ==  '500₺ - 999₺')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] == '0 - 499₺')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] == '-')
                                ), 'Baska Kurumdan Aldigi Burs Miktari'] = df.loc[
                        ~((df['Baska Kurumdan Aldigi Burs Miktari'] == '1000₺ ve üstü')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] ==  '500₺ - 999₺')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] == '0 - 499₺')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] == '-')
                                ), 'Baska Kurumdan Aldigi Burs Miktari'].str.findall('\d+').fillna("-")

In [ ]:
df.drop(df.loc[
                        ~((df['Baska Kurumdan Aldigi Burs Miktari'] == '1000₺ ve üstü')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] ==  '500₺ - 999₺')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] == '0 - 499₺')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] == '-'))
                              & (df['Baska Kurumdan Aldigi Burs Miktari'].str.len() == 0)
                                , 'Baska Kurumdan Aldigi Burs Miktari'].index, inplace=True)

In [ ]:
#df.loc[df.id == 57067, 'Baska Kurumdan Aldigi Burs Miktari']

In [ ]:
df.loc[
                        ~((df['Baska Kurumdan Aldigi Burs Miktari'] == '1000₺ ve üstü')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] ==  '500₺ - 999₺')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] == '0 - 499₺')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] == '-')
                                ), 'Baska Kurumdan Aldigi Burs Miktari'] = df.loc[
                        ~((df['Baska Kurumdan Aldigi Burs Miktari'] == '1000₺ ve üstü')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] ==  '500₺ - 999₺')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] == '0 - 499₺')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] == '-')
                                ), 'Baska Kurumdan Aldigi Burs Miktari'].apply(lambda x: '-' if x == '0' else max(x))

In [ ]:
df.drop(df.loc[
                        ~((df['Baska Kurumdan Aldigi Burs Miktari'] == '1000₺ ve üstü')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] ==  '500₺ - 999₺')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] == '0 - 499₺')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] == '-'))
                              & (df['Baska Kurumdan Aldigi Burs Miktari'].str.len() == 0)
                                , 'Baska Kurumdan Aldigi Burs Miktari'].index, inplace=True)

In [ ]:
#TÜFE değrlerine göre 2023 baz alınarak tahmini enflasyon
tufe=pd.DataFrame(
    {
        'yil': [2014,2015,2016,2017,2018,2019,2020,2021,2022],
        'tufe': [5.17,4.81,4.49,4.02,3.46,3.02,2.70,2.34,1.58]
    }
)

tufe['yil'] = pd.to_datetime(tufe['yil'], format='%Y')
tufe

In [ ]:
for a in tufe.index:
    df.loc[
                        ~((df['Baska Kurumdan Aldigi Burs Miktari'] == '1000₺ ve üstü')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] ==  '500₺ - 999₺')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] == '0 - 499₺')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] == '-'))
                              & (df['Basvuru Yili'].dt.year == tufe.iloc[a].yil.year)
                                 , 'Baska Kurumdan Aldigi Burs Miktari'] = df.loc[
                        ~((df['Baska Kurumdan Aldigi Burs Miktari'] == '1000₺ ve üstü')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] ==  '500₺ - 999₺')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] == '0 - 499₺')
                              | (df['Baska Kurumdan Aldigi Burs Miktari'] == '-'))
                              & (df['Basvuru Yili'].dt.year == tufe.iloc[a].yil.year)
                                 , 'Baska Kurumdan Aldigi Burs Miktari'].apply(lambda x: tufe.iloc[a].tufe* int(x)).apply(lambda x: '-' if int(x) <= 50 else (
                                                                               '0 - 499₺' if (int(x) > 10) & (int(x) <= 500) else (
                                                                                   '500₺ - 999₺' if (int(x) > 500) & (int(x) < 1000) else '1000₺ ve üstü')
                                                                               ))

In [ ]:
df['Baska Kurumdan Aldigi Burs Miktari'].unique()

In [ ]:
df.isnull().sum()

In [ ]:
df['Baska Bir Kurumdan Burs Aliyor mu?'] = df['Baska Bir Kurumdan Burs Aliyor mu?'].fillna('hayır')
df['Baska Bir Kurumdan Burs Aliyor mu?'].unique()

## df2 ile devam edilecek

In [ ]:
df2 = df.copy()
df2

In [ ]:
pd.to_datetime(df_raw['Basvuru Yili'], format='%Y').dt.year.unique()

In [ ]:
import matplotlib.pyplot as plt
df_raw_basvuru_yili_datetime = pd.to_datetime(df_raw['Basvuru Yili'], format='%Y')
for i in df_raw_basvuru_yili_datetime.dt.year.unique():
        print(f'Basvuru Yili: {i}-----------------------------')
        plt.title(f"{i} yılı kategorilere göre boş veriler")
        plt.xlabel("Kategori")
        plt.ylabel("Veri Miktarı")
        plt.xticks(rotation = 90)
        #plt.figure(figsize=(6, 6))
        plt.bar(df_raw.columns, 
                len(df_raw.loc[df_raw_basvuru_yili_datetime.dt.year== i])-df_raw.loc[df_raw_basvuru_yili_datetime.dt.year == i, df_raw.columns].isna().sum(),
                width=0.8
               )
    
        plt.bar(df_raw.columns,
                df_raw.loc[df_raw_basvuru_yili_datetime.dt.year == i, df_raw.columns].isna().sum(),
                bottom= len(df_raw.loc[df_raw_basvuru_yili_datetime.dt.year== i])-df_raw.loc[df_raw_basvuru_yili_datetime.dt.year == i, df_raw.columns].isna().sum(),
                color='red',
                width=0.8
                )
        plt.show()
            #print(j,':', df_raw.loc[df_raw_basvuru_yili_datetime.dt.year == i, j].isna().sum())
                

In [ ]:
for i in df_raw["Basvuru Yili"].unique():
    sns.displot(data=df_raw[df_raw["Basvuru Yili"]==i],x="Degerlendirme Puani",)
    plt.title(i)

In [ ]:
import seaborn as sns
sns.displot(data=df_raw,x="Degerlendirme Puani", hue="Basvuru Yili",kind="kde")

In [ ]:
df2.loc[(df2["Basvuru Yili"]==2016)].shape()

In [ ]:
#numeric categoric
def capture_cols(dataframe, cat_th=10, car_th=20):

    cat_cols = [col for col in dataframe.columns if str(dataframe[col].dtypes) in ["category", "object", "bool"]]

    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and dataframe[col].dtypes in ["int", "float"]]

    cat_but_car = [col for col in dataframe.columns if
                   dataframe[col].nunique() > car_th and str(dataframe[col].dtypes) in ["category", "object"]]

    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes in ["int", "float", "<M8[ns]"]]
    num_cols = [col for col in num_cols if col not in cat_cols]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')

    return cat_cols, num_cols, cat_but_car

cat_cols, num_cols, cat_but_car = capture_cols(df2)

In [ ]:
numeric = df_total[n_cols]
lof = LocalOutlierFactor(n_neighbors=81)
lof.fit_predict(numeric)

df_scores = lof.negative_outlier_factor_
df_scores[0:5]
np.sort(df_scores)[0:5]

scores = pd.DataFrame(np.sort(df_scores))
scores.plot(stacked=True, xlim=[0,50], style='.-')
plt.show()

### ____________________________________
  < burdan öncesini elleşme sal bana >
   ------------------------------------
          \   ^__^
           \  (oo)\_______
              (__)\       )\/\
                  ||----w |
                  ||     ||

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV, validation_curve
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import StandartScaler, OneHotEncoder, LabelEncoder

In [2]:
# label encoding: Universite Kacinci Sinif, Universite Not Ortalaması, Lise Mezuniyet Notu, Baska Kurumdan Aldigi Burs Miktari, Anne Egitim Durumu, Baba Egitim Durumu 
# Geriye kalan cot_cols one hot encoding
label_enc_cols = ["Universite Kacinci Sinif", "Universite Not Ortalaması", 
                  "Lise Mezuniyet Notu", "Baska Kurumdan Aldigi Burs Miktari", 
                  "Anne Egitim Durumu", "Baba Egitim Durumu"]

def label_encoder(dataframe, le_col):
  le = LabelEncoder()
  dataframe[bin_col] = le.fit_transform(dataframe[bin_col])
  return dataframe
    
#df2 = label_encoder(df2, label_enc_cols)

def one_hot_encoder(dataframe, ohe_col):
    ohe = OneHotEncoder()
    dataframe[ohe_col] = ohe.fit_transform(dataframe[ohe_col])

other_cols = df2.columns.difference(label_enc_cols)
other_cols

NameError: name 'df2' is not defined

In [ ]:
X = df2.drop(['Degerlendirme Puani'], axis=1)
y = df2['Degerlendirme Puani']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
models = {
    'Linear Regression': LinearRegression(),
    'Gradient Boosting Regressor': GradientBoostingRegressor(random_state=17),
    'Random Forest Regressor': RandomForestRegressor(random_state=17),
    'Extra Trees Regressor': ExtraTreesRegressor(random_state=17),
    'AdaBoost Regressor': AdaBoostRegressor(random_state=17),
    'Decision Tree Regressor': DecisionTreeRegressor(random_state=17),
    'Extreme Gradient Boosting': XGBRegressor(random_state=17),
    'Light Gradient Boosting Machine': LGBMRegressor(random_state=17),
    'CatBoost Regressor': CatBoostRegressor(random_state=17),
}

params = {
    "Gradient Boosting Regressor": {
        "learning_rate": [0.01],
        "max_depth": [3, 5, 8],
        "max_features": ["auto", "sqrt", "log2"],
        "n_estimators": [100, 200, 400, 700],
        "subsample": [1, 0.8]
    },
    "Random Forest Regressor": {
        "max_depth": [3, 5, 8],
        "max_features": ["auto", "sqrt", "log2"],
        "min_samples_split": [2, 5, 10],
        "n_estimators": [100, 200, 400, 700]
    },
    "Extra Trees Regressor": {
        "n_estimators": [100, 200, 400, 700],
        "max_depth": [3, 5, 8],
        "min_samples_split": [2, 5, 10],
        "max_features": ["auto", "sqrt", "log2"],
    },
    "AdaBoost Regressor": {
        "learning_rate": [0.01],
        "n_estimators": [100, 200, 400, 700],
        "loss": ['linear', 'square', 'exponential']
    },
    "Decision Tree Regressor": {
        "max_depth": [3, 5, 8],
        "min_samples_split": [2, 5, 10],
        "max_features": ["auto", "sqrt", "log2"],
    },
    "Extreme Gradient Boosting": {
        "colsample_bytree": [1, 0.7],
        "learning_rate": [0.01],
        "max_depth": [3, 5, 8],
        "n_estimators": [100, 200, 400, 700],
        "subsample": [1, 0.8]
    },
    "Light Gradient Boosting Machine": {
        "colsample_bytree": [1, 0.7],
        "learning_rate": [0.01],
        "max_depth": [3, 5, 8],
        "n_estimators": [100, 200, 400, 700],
        "num_leaves": [3, 7]
    },
    "CatBoost Regressor": {
        "learning_rate": [0.01],
        "max_depth": [3, 5, 8],
        "n_estimators": [100, 200, 400, 700],
        "subsample": [1, 0.8]
    }
}

def train(models, X, y, test_size=0.2, cv=5):

  results = {}
  params = {}

  for i, (model_name, model) in enumerate(models.items()):

    params[model_name] = model.get_params()

    scoring = {
        "neg_mean_squared_error": make_scorer(mean_squared_error),
    }

    results[model_name] = {"RMSE": np.mean(np.sqrt(-cross_val_score(model, X, y, cv=cv, scoring="neg_mean_squared_error")))}

  return results, params
        
def training_results(results):

  results_df = pd.DataFrame(results).T

  return results_df

results, params = train(models, params, X, y)
training_results(results)

In [ ]:
def hyperparameter_tuning(models, params, X, y, cv=5, random_state=17):

  results = {}
  best_model = None
  best_score = 0

  for model_name, model in models.items():

    if model_name not in params:
      print(f"{model_name} does not require hyperparameters.")
      continue

    print(f"Performing Hyperparameter Tuning for {model_name}...")

    gs_cv = GridSearchCV(model, params[model_name], cv=cv, n_jobs=-1, verbose=False).fit(X, y)
    gs_final = model.set_params(**gs_cv.best_params_, random_state=random_state).fit(X, y)

    best_params = gs_cv.best_params_
    score = gs_cv.best_score_

    if score > best_score:
      best_score = score
      best_model = gs_final

    scoring = {
        "neg_mean_squared_error": make_scorer(mean_squared_error)
    }

    results[model_name] = {
            'Best Parameters': best_params,
            "RMSE": np.mean(np.sqrt(-cross_val_score(model, X, y, cv=cv, scoring="neg_mean_squared_error")))
    }

    print(f"{model_name} - Best Parameters: {best_params}")

  return results, gs_final, best_model

hyper_results, gs_final, best_model = hyperparameter_tuning(models, params, X, y)
print(f"\nThe model with the best score is {best_model}\n")
train_results(hyper_results)

In [ ]:
def plot_learning_curve(estimator, title, X, y, train_sizes = np.linspace(.1, 1.0, 5), scoring="neg_mean_squared_error"):
    
  train_sizes, train_scores, test_scores = learning_curve(gs_final, X, y,
                                                          cv=5, scoring=scoring,
                                                          n_jobs=-1, train_sizes=train_sizes)
  plt.figure()
  plt.title(title)

  plt.xlabel('Number of Training Samples')
  plt.ylabel('Score')

  train_scores_mean = np.mean(train_scores, axis=1)
  train_scores_std = np.std(train_scores, axis=1)
  test_scores_mean = np.mean(test_scores, axis=1)
  test_scores_std = np.std(test_scores, axis=1)

  plt.grid()

  plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                   train_scores_mean + train_scores_std, alpha=0.1,
                   color="r")

  plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                   test_scores_mean + test_scores_std, alpha = 0.1, color = "g")

  plt.plot(train_sizes, train_scores_mean, 'o-', color = "r",
           label = "Training score")

  plt.plot(train_sizes, test_scores_mean, 'o-', color = "g",
           label = "Cross-validation score")

  plt.legend(loc = "best")
  plt.show

plot_learning_curve(best_model, "Learning Curve For Best Model", x, y)

In [55]:
!git add .
!git commit -m "berke<3doa"
!git push

[main ca0ec46] berke<3doa
 1 file changed, 11 insertions(+), 10 deletions(-)
Nesneler ortaya dökülüyor: 34, bitti.
Nesneler sayılıyor: 100% (34/34), bitti.
Delta sıkıştırması 8 iş parçacığı kullanıyor
Nesneler sıkıştırılıyor: 100% (26/26), bitti.
Nesneler yazılıyor: 100% (29/29), 8.20 MiB | 3.53 MiB/sn, bitti.
Toplam 29 (delta 16), yeniden kullanılan 0 (delta 0), yeniden kullanılan paket 0 (0 konumundan)
remote: Resolving deltas: 100% (16/16), completed with 4 local objects.
remote: error: Trace: f96d55a64b279c32ccf328184706a0e6ef4bf4c86dee484449f368b55b833802
remote: error: See https://gh.io/lfs for more information.
remote: error: File .ipynb_checkpoints/nanatolia_btk_datahon-checkpoint.ipynb is 175.81 MB; this exceeds GitHub's file size limit of 100.00 MB
remote: error: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.
To https://github.com/DALLI-KAKTUS/btk-datathon-2024.git
 ! [remote rejected] main -> main (pre-receive hook decli